In [1]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename='/data/mine/vadeec/merged/ukdale.h5',
        buildings=[1, 2],
        sample_period=6,
        window=("2013-04-01", "2013-05-01")
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

Loading kettle for UK-DALE_building_1...


INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...


Loaded 164 kettle activations from UK-DALE_building_1.


INFO:neuralnilm.data.loadactivations:Loaded 164 kettle activations from UK-DALE_building_1.


Loading microwave for UK-DALE_building_1...


...

INFO:neuralnilm.data.loadactivations:Loaded 0 microwave activations from UK-DALE_building_2.


Loading washing machine for UK-DALE_building_2...


INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_2...


Loaded 0 washing machine activations from UK-DALE_building_2.


INFO:neuralnilm.data.loadactivations:Loaded 0 washing machine activations from UK-DALE_building_2.


In [3]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance='kettle',
    seq_length=2048
)

In [4]:
seq = source.get_sequence(enable_all_appliances=True)

In [5]:
all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

True

In [6]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#plt.show()

In [7]:
sample = source.get_batch(num_seq_per_batch=1024)

In [8]:
pipeline = DataPipeline(
    source, 
    num_seq_per_batch=64,
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

In [9]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'input_processing': [],
  'num_seq_per_batch': 64,
  'source': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
    'allow_incomplete_target': True,
    'distractor_inclusion_prob': 1.0,
    'include_incomplete_target_in_output': True,
    'issued_validation_warning': False,
    'rng_seed': None,
    'seq_length': 2048,
    'target_appliance': 'kettle',
    'target_inclusion_prob': 1.0,
    'uniform_prob_of_selecting_each_model': True}},
  'target_processing': [{'DivideBy': {'divisor': 296.15518}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'buildings': [1, 2],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'window': ('2013-04-01', '2013-05-01')}}}

In [10]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {'dense_layer_0': 100}
    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [11]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [13]:
from neuralnilm.trainer import Trainer

trainer = Trainer(
    net,
    data_pipeline=pipeline,
    repeat_callbacks=[
        (  10, Trainer.validate)
    ]
)

In [14]:
trainer.fit(50)

Compiling train cost function...


INFO:neuralnilm.trainer:Compiling train cost function...


Done compiling cost function.


INFO:neuralnilm.trainer:Done compiling cost function.


Compiling validation cost function...


INFO:neuralnilm.trainer:Compiling validation cost function...


Done compiling cost function.


INFO:neuralnilm.trainer:Done compiling cost function.


In [15]:
trainer.iteration

50

In [16]:
trainer.training_costs

[37807.317017981302,
 0.96741127529235937,
 1.0100021994050403,
 0.94475077102432914,
 0.98087564983746589,
 447.18027682016844,
 1.0067202897422904,
 1.0633125505810006,
 0.98060758520681102,
 1.0036914478153074,
 0.93800780851490539,
 1.1101724582181778,
 1.0171553836165979,
 0.89902630663280803,
 0.92780575891668393,
 1.0234892330285847,
 0.99704747906739444,
 1.0401542642350994,
 1.1054850625109667,
 0.91873398557033148,
 0.99562400284304176,
 0.93642272801951287,
 1.0286516854315457,
 1.0747585250121574,
 0.95229736623896899,
 0.91441199040630461,
 0.9695728470643098,
 0.91543073964198363,
 1.05437153050264,
 0.94878400981222899,
 1.014345340938178,
 0.87521737876593109,
 1.0167435907778901,
 1.0374977118267339,
 0.89801007287511769,
 1.0053220690103177,
 1.0738991046966011,
 1.0381086716416614,
 1.0107669706461053,
 1.034773520838439,
 1.0896323442895557,
 0.90449232714541339,
 1.0021238946061193,
 1.0640720490511886,
 0.98415512623685586,
 1.0032412830574671,
 0.9780832986315179

In [19]:
trainer.validation_costs[0].flatten()[0]

0.89424162130047669

In [20]:
trainer.validation_costs[0]

array(0.8942416213004767)